In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier

In [2]:
data = pd.read_csv('winequality-red.csv')
data.columns = [col.strip() for col in data.columns]

In [3]:
old_data_length = len(data)
IF_model = IsolationForest(contamination=0.1) # Remove 10% data
outliers = IF_model.fit_predict(data) # train IF model
data['outliers'] = outliers

# Remove outliers
data_outliers = data[data['outliers'] == -1]
new_data = data[data['outliers'] != -1].drop(columns = ['outliers'])

new_data_len = len(new_data)

print(f"There are total {old_data_length - new_data_len} rows been removied.")

There are total 160 rows been removied.


In [4]:
def wine_classifier(quality, top_percent):
    if quality >= top_percent[0.9]:
        return 'Great'
    elif quality >= top_percent[0.7]:
        return 'Good'
    elif quality >= top_percent[0.3]:
        return 'Normal'
    else:
        return 'Bad'

In [5]:
top_percent = new_data['quality'].quantile([0.3, 0.7, 0.9])

quality = []
for index, row in new_data.iterrows():
    quality.append(wine_classifier(row['quality'], top_percent))

new_data['classified_quality'] = quality

In [6]:
x = new_data.drop(columns=['quality', 'classified_quality'], axis=1)
y = OneHotEncoder().fit_transform(new_data[['classified_quality']]).toarray()

In [7]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=10)

In [8]:
from sklearn.preprocessing import MinMaxScaler
minmax_scaler = MinMaxScaler()
scale_xTrain = minmax_scaler.fit_transform(xTrain)
scale_xTest = minmax_scaler.transform(xTest)

In [9]:
#change y arrays to int arrays
int_yTrain = yTrain.astype(int)
int_yTest = yTest.astype(int)

#create NN with input layer that takes in 11 features. Has 6 hidden layers, 512, 256, 128, 64, 32, and 4 notes respectively. 
#All have 'relu' activation functions except for last layer, which uses 'softmax'
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(11,)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(4, activation="softmax") 
    ])

#compile model using the adam optimization algo, using categorical crossentropy since we have a multiclass model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

epochs = 50
batch = 32
modelfit = model.fit(scale_xTrain, int_yTrain, epochs=epochs, batch_size=batch)

test_loss, test_accuracy = model.evaluate(scale_xTest, yTest)

Epoch 1/50
29/29 [==============================] - 1s 8ms/step - loss: 1.1575 - accuracy: 0.4620 - val_loss: 1.0156 - val_accuracy: 0.5411
Epoch 2/50
29/29 [==============================] - 0s 3ms/step - loss: 0.9573 - accuracy: 0.5707 - val_loss: 0.8948 - val_accuracy: 0.6104
Epoch 3/50
29/29 [==============================] - 0s 4ms/step - loss: 0.9047 - accuracy: 0.5826 - val_loss: 0.8718 - val_accuracy: 0.6190
Epoch 4/50
29/29 [==============================] - 0s 3ms/step - loss: 0.8770 - accuracy: 0.6109 - val_loss: 0.9053 - val_accuracy: 0.5931
Epoch 5/50
29/29 [==============================] - 0s 3ms/step - loss: 0.8664 - accuracy: 0.6120 - val_loss: 0.8499 - val_accuracy: 0.6147
Epoch 6/50
29/29 [==============================] - 0s 3ms/step - loss: 0.8428 - accuracy: 0.6337 - val_loss: 0.9159 - val_accuracy: 0.5974
Epoch 7/50
29/29 [==============================] - 0s 3ms/step - loss: 0.8732 - accuracy: 0.6065 - val_loss: 0.8465 - val_accuracy: 0.6277
Epoch 8/50
29/29 [==

In [10]:

# def create_model(optimizer='adam', kernel_initializer='glorot_uniform', dropout_rate=0.2):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Input(shape=(11,)),
#         tf.keras.layers.Dense(512, activation="relu", kernel_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(256, activation="relu", kernel_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(128, activation="relu", kernel_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(64, activation="relu", kernel_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(32, activation="relu", kernel_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(4, activation="softmax", kernel_initializer='glorot_uniform') 
#     ])

#     model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
#     return model

# model = KerasClassifier(build_fn = create_model, verbose = 0, kernel_initializer = 'glorot_uniform')
# epochs = 50
# batch = 32
# modelfit = model.fit(scale_xTrain, int_yTrain, epochs=epochs, batch_size=batch, validation_split=0.2)


In [16]:
#create wrapper in order to format keras for scikit-learn, trying glorot uniform  this time
def create_model(optimizer='adam', kernel_initializer='glorot_uniform', dropout_rate=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(11,)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(4, activation="softmax") 
    ])

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

model = KerasClassifier(build_fn = create_model, verbose = 0, kernel_initializer = 'glorot_uniform', dropout_rate=0.4)
epochs = 50
batch = 32
history = model.fit(scale_xTrain, int_yTrain, epochs=epochs, batch_size=batch)

#testing adam, rmsprop, or sgd for best optimizer, as well as best batch size and number of epochs.
hparam = {'epochs': [36, 50, 100], 'batch_size': [32,64], 'dropout_rate': [0.2,0.3,0.4], 'optimizer': ['adam', 'rmsprop', 'SGD'], 'kernel_initializer': ['glorot_uniform', 'normal']}

random_search = RandomizedSearchCV(
    model,
    param_distributions = hparam,
    n_iter = 10,
    cv = 3,
    scoring = 'accuracy',
    verbose = 2

)


random_search.fit(scale_xTrain, int_yTrain)
print("Best Hyperparameters:", random_search.best_params_)

C:\Users\Tristan H\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


C:\Users\Tristan H\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV] END batch_size=64, dropout_rate=0.2, epochs=36, kernel_initializer=glorot_uniform, optimizer=adam; total time=   1.8s


C:\Users\Tristan H\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

[CV] END batch_size=64, dropout_rate=0.3, epochs=36, kernel_initializer=glorot_uniform, optimizer=SGD; total time=   1.7s


C:\Users\Tristan H\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


[CV] END batch_size=64, dropout_rate=0.3, epochs=36, kernel_initializer=glorot_uniform, optimizer=SGD; total time=   2.1s


C:\Users\Tristan H\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Best Hyperparameters: {'optimizer': 'rmsprop', 'kernel_initializer': 'glorot_uniform', 'epochs': 50, 'dropout_rate': 0.3, 'batch_size': 64}


In [20]:
#implementing model with best hyperparameters
model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(11,)),
        tf.keras.layers.Dense(512, activation="relu",kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(256, activation="relu",kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(128, activation="relu",kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(64, activation="relu",kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(32, activation="relu",kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(4, activation="softmax",kernel_initializer='glorot_uniform') 
    ])

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

epochs = 50
batch = 64
history = model.fit(scale_xTrain, int_yTrain, epochs=epochs, batch_size=batch)

test_loss, test_accuracy = model.evaluate(scale_xTest, int_yTest)

Epoch 1/50
18/18 [==============================] - 1s 3ms/step - loss: 1.1532 - accuracy: 0.4579
Epoch 2/50
18/18 [==============================] - 0s 3ms/step - loss: 0.9772 - accuracy: 0.5387
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 0.9223 - accuracy: 0.5743
Epoch 4/50
18/18 [==============================] - 0s 3ms/step - loss: 0.9115 - accuracy: 0.5795
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 0.8855 - accuracy: 0.5995
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 0.8710 - accuracy: 0.6177
Epoch 7/50
18/18 [==============================] - 0s 2ms/step - loss: 0.8717 - accuracy: 0.6012
Epoch 8/50
18/18 [==============================] - 0s 2ms/step - loss: 0.8729 - accuracy: 0.6056
Epoch 9/50
18/18 [==============================] - 0s 2ms/step - loss: 0.8719 - accuracy: 0.6142
Epoch 10/50
18/18 [==============================] - 0s 2ms/step - loss: 0.8367 - accuracy: 0.6386
Epoch 11/50
18/18 [